## ダミーの保険契約データからソーシャルグラフの関係性を抽出
ダミー個人情報の取得元 https://tm-webtools.com/Tools/TestData

ダミー住所情報の取得元 http://jusyo.jp/

In [2]:
import pandas as pd

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

In [3]:
df_branch = pd.read_csv('data/branch_info.csv')
df_staff = pd.read_csv('data/generated_staffs.csv')
df_contract = pd.read_csv('data/generated_contracts.csv')

In [4]:
df_branch.head(3)

,postal_area,post_office_cd,post_office_name,post_office_pronunciation
0,神田,1008,神田郵便局,かんだ
1,神田,1024,小川町駅前郵便局,おがわまちえきまえ
2,神田,1042,西神田郵便局,にしかんだ


In [5]:
df_staff.head(3)

,staff_cd,last_name,first_name,last_name_pronunciation,first_name_pronunciation,gender,birthday,age,phone,post_office_cd
0,0,柳,昭一,ヤナギ,ショウイチ,男,1985/9/25,34,090-3602-0717,755
1,1,栗原,昌之,クリハラ,マサユキ,男,1985/8/29,34,090-2226-4253,1128
2,2,戸川,心愛,トガワ,ココア,女,1969/10/21,50,080-8104-8561,1072


In [6]:
df_contract.head(3)

,staff_cd,type,contractor_last_name,contractor_last_name_pron,contractor_first_name,contractor_first_name_pron,contractor_age,contractor_birthday,contractor_gender,contractor_postal_cd,...,insured_first_name,insured_first_name_pron,insured_age,insured_birthday,insured_gender,insured_postal_cd,insured_address_1,insured_address_2,insured_address_3,insured_phone
0,17,定期,江崎,エサキ,実桜,ミオ,15,2004/8/25,女,170-8457,...,実桜,ミオ,15,2004/8/25,女,170-8457,東京都,豊島区,南大塚２丁目２９番７号,090-8967-9720
1,17,養老,江崎,エサキ,実桜,ミオ,15,2004/8/25,女,170-8457,...,実桜,ミオ,15,2004/8/25,女,170-8457,東京都,豊島区,南大塚２丁目２９番７号,090-8967-9720
2,17,学資,江崎,エサキ,実桜,ミオ,15,2004/8/25,女,170-8457,...,裕子,ヒロコ,0,1987/3/24,女,170-8457,東京都,豊島区,南大塚２丁目２９番７号,080-6117-0321


In [7]:
post_office_cds = [str(x) for x in df_staff.post_office_cd.unique().tolist()]
df_target_branch = df_branch[df_branch.post_office_cd.isin(post_office_cds)]
df_target_branch

,postal_area,post_office_cd,post_office_name,post_office_pronunciation
4,神田,1128,神田今川橋郵便局,かんだいまがわばし
29,銀座,1072,東京高等裁判所内郵便局,とうきょうこうとうさいばんしょない
46,銀座,755,千代田霞が関郵便局,ちよだかすみがせき


### 郵便局ノードを生成
今回作成するネットワークの親ノードになる郵便局データを作成する

In [8]:
# データに含まれる郵便局の件数を取得
num_post_office = df_target_branch.shape[0]

# ノードのID（グラフ全体でユニークである必要あり）を生成する
ids = ['po{}'.format(x) for x in df_target_branch.post_office_cd.tolist()]

# ノードに付ける属性（プロパティ）として郵便局名を抽出する
names = [x for x in df_target_branch.post_office_name.tolist()]

# csv出力のため、辞書型に変換する
postoffice_dict = {'~id': ids, 'name:String': names, '~label': 'post_office'}

# Pandasの機能を利用してcsvとして出力する
df_node_postoffice = pd.DataFrame(postoffice_dict)
df_node_postoffice.to_csv('data/node1.csv', index=False, encoding='utf-8')

In [9]:
df_node_postoffice

,~id,name:String,~label
0,po1128,神田今川橋郵便局,post_office
1,po1072,東京高等裁判所内郵便局,post_office
2,po755,千代田霞が関郵便局,post_office


### 社員ノードを生成

In [10]:
df_staff.head(3)

,staff_cd,last_name,first_name,last_name_pronunciation,first_name_pronunciation,gender,birthday,age,phone,post_office_cd
0,0,柳,昭一,ヤナギ,ショウイチ,男,1985/9/25,34,090-3602-0717,755
1,1,栗原,昌之,クリハラ,マサユキ,男,1985/8/29,34,090-2226-4253,1128
2,2,戸川,心愛,トガワ,ココア,女,1969/10/21,50,080-8104-8561,1072


In [11]:
# データに含まれる郵便局の件数を取得
num_staffs = df_staff.shape[0]

# 姓と名を合体させて氏名カラムを生成
df_staff['name'] = df_staff['last_name'] + df_staff['first_name']

# ノードのID（グラフ全体でユニークである必要あり）を生成する
ids = ['s{}'.format(x) for x in df_staff.staff_cd.tolist()]

# ノードに付ける属性（プロパティ）として社員の名前と年齢、性別を抽出する
names = [x for x in df_staff.name.tolist()]
ages = [x for x in df_staff.age.tolist()]
genders = [x for x in df_staff.gender.tolist()]

# 生成したデータをPandas DataFrameに変換してcsv出力する
df_node_staff = pd.DataFrame({'~id': ids, 'name:String': names, 'age:Int': ages, 'gender:String': genders, '~label': 'staff'})
df_node_staff.to_csv('data/node2.csv', index=False, encoding='utf-8')

In [12]:
df_node_staff.head(3)

,~id,name:String,age:Int,gender:String,~label
0,s0,柳昭一,34,男,staff
1,s1,栗原昌之,34,男,staff
2,s2,戸川心愛,50,女,staff


### 郵便局->社員のエッジを生成

In [13]:
# エッジを生成
edge_ids = ['po_s{}'.format(x) for x in range(0, num_staffs)]
from_nodes = ['po{}'.format(x) for x in df_staff.post_office_cd.tolist()]
to_nodes = ['s{}'.format(x) for x in df_staff.staff_cd.tolist()]

df_edge = pd.DataFrame({'~id': edge_ids, '~from': from_nodes, '~to': to_nodes, '~label': 'branch_staff'})
df_edge.to_csv('data/edge1.csv', index=False)

In [14]:
df_edge.head(3)

,~id,~from,~to,~label
0,po_s0,po755,s0,branch_staff
1,po_s1,po1128,s1,branch_staff
2,po_s2,po1072,s2,branch_staff


### 契約者ノードを生成

In [16]:
contractor_columns = ['contractor_last_name',
                      'contractor_last_name_pron',
                      'contractor_first_name',
                      'contractor_first_name_pron',
                      'contractor_age',
                      'contractor_birthday',
                      'contractor_gender',
                      'contractor_postal_cd',
                      'contractor_address_1',
                      'contractor_address_2',
                      'contractor_address_3',
                      'contractor_phone']
contractor_unique_cols = ['contractor_last_name',
                      'contractor_first_name',
                      'contractor_birthday',
                      'contractor_address_1',
                      'contractor_address_2',
                      'contractor_address_3']

保険契約者の名寄せを実行

In [17]:
# ユニークキーとして利用する列だけを抽出
df_unique_contractor = df_contract[contractor_columns].copy()

# 重複するレコードをグループ化して1レコードに集約する
df_unique_contractor = df_unique_contractor.groupby(by=contractor_unique_cols).max()

# 重複排除したレコードに対してIDを割り振る
df_unique_contractor['contractor_id'] = ['c{}'.format(x) for x in range(0, df_unique_contractor.shape[0])]

# グループ化に利用した列を索引からデータ列に取り出し
df_unique_contractor.reset_index(inplace=True)

In [18]:
# 保険契約者のノードを生成
df_unique_contractor['contractor_name'] = df_unique_contractor['contractor_last_name'] + df_unique_contractor['contractor_first_name']

ids = [x for x in df_unique_contractor.contractor_id.tolist()]
names = [x for x in df_unique_contractor.contractor_name.tolist()]
ages = [x for x in df_unique_contractor.contractor_age.tolist()]
genders = [x for x in df_unique_contractor.contractor_gender.tolist()]

df_node_contractor = pd.DataFrame({'~id': ids, 'name:String': names, 'age:Int': ages, 'gender:String': genders, '~label': '契約者'})

### 被保険者ノードを生成

In [19]:
insured_columns = ['insured_last_name',
                      'insured_last_name_pron',
                      'insured_first_name',
                      'insured_first_name_pron',
                      'insured_age',
                      'insured_birthday',
                      'insured_gender',
                      'insured_postal_cd',
                      'insured_address_1',
                      'insured_address_2',
                      'insured_address_3',
                      'insured_phone']
insured_unique_cols = ['insured_last_name',
                      'insured_first_name',
                      'insured_birthday',
                      'insured_address_1',
                      'insured_address_2',
                      'insured_address_3']

In [20]:
# 被保険者の名寄せを実行
df_unique_insured = df_contract[insured_columns].copy()
df_unique_insured = df_unique_insured.groupby(by=insured_unique_cols).max()
df_unique_insured['insured_id'] = ['i{}'.format(x) for x in range(0, df_unique_insured.shape[0])]
df_unique_insured.reset_index(inplace=True)

In [21]:
# 保険契約者のノードを生成
df_unique_insured['insured_name'] = df_unique_insured['insured_last_name'] + df_unique_insured['insured_first_name']

ids = [x for x in df_unique_insured.insured_id.tolist()]
names = [x for x in df_unique_insured.insured_name.tolist()]
ages = [x for x in df_unique_insured.insured_age.tolist()]
genders = [x for x in df_unique_insured.insured_gender.tolist()]

df_node_insured = pd.DataFrame({'~id': ids, 'name:String': names, 'age:Int': ages, 'gender:String': genders, '~label': '被保険者'})

In [22]:
# 保険契約者と被保険者のデータを一つのDataFrameに接続
df_node_out = pd.concat([df_node_contractor, df_node_insured])

# ロード用ファイルとして出力
df_node_out.to_csv('data/node3.csv', index=False, encoding='utf-8')

### 社員->契約者エッジを生成

In [23]:
# 社員と契約者（のべ）の対応表を生成
df_staff_contractor = pd.merge(df_contract, df_unique_contractor, how='left', on=contractor_unique_cols)
df_staff_contractor = df_staff_contractor[['staff_cd', 'contractor_id', 'type']].copy()

In [24]:
# エッジを生成
edge_ids = ['s_c{}'.format(x) for x in df_staff_contractor.index.tolist()]
from_nodes = ['s{}'.format(x) for x in df_staff_contractor.staff_cd.tolist()]
to_nodes = [x for x in df_staff_contractor.contractor_id.tolist()]
labels = ['契約：{}'.format(x) for x in df_staff_contractor.type.tolist()]

df_sc_edge = pd.DataFrame({'~id': edge_ids, '~from': from_nodes, '~to': to_nodes, '~label': labels})
df_sc_edge.to_csv('data/edge2.csv', index=False)

### 契約者->被保険者エッジを生成

In [25]:
# 契約者と被保険者の対応表を生成
df_temp = pd.merge(df_contract, df_staff_contractor.contractor_id, how='inner', left_index=True, right_index=True)
df_contractor_insured = pd.merge(df_temp, df_unique_insured, how='left', on=insured_unique_cols)

In [26]:
def detect_relation(row):
    if (row.contractor_last_name == row.insured_last_name) and (row.contractor_first_name == row.insured_first_name):
        return '被保険：本人'
    elif (row.contractor_last_name == row.insured_last_name) and (row.contractor_first_name != row.insured_first_name):
        return '被保険：家族'
    else:
        return '被保険：第3者'
        
detect_relation(df_contractor_insured.iloc[1])

'被保険：本人'

In [27]:
# エッジを生成
edge_ids = ['c_i{}'.format(x) for x in df_contractor_insured.index.tolist()]
from_nodes = [x for x in df_contractor_insured.contractor_id.tolist()]
to_nodes = [x for x in df_contractor_insured.insured_id.tolist()]
labels = df_contractor_insured.apply(detect_relation, axis=1).tolist()

df_ci_edge = pd.DataFrame({'~id': edge_ids, '~from': from_nodes, '~to': to_nodes, '~label': labels})
df_ci_edge.to_csv('data/edge3.csv', index=False)

### 生成したノードとエッジの情報をS3にコピー

"&lt;S3 bucket name&gt;" をハンズオン環境ごとのバケット名で置き換える  
CloudFormationのサービス画面からスタック「vpc」→出力タブで、ハンズオン環境のバケット名が確認できる  

In [ ]:
!aws s3 cp 'data/node1.csv' 's3://<S3 bucket name>/data/graph/'
!aws s3 cp 'data/node2.csv' 's3://<S3 bucket name>/data/graph/'
!aws s3 cp 'data/node3.csv' 's3://<S3 bucket name>/data/graph/'
!aws s3 cp 'data/edge1.csv' 's3://<S3 bucket name>/data/graph/'
!aws s3 cp 'data/edge2.csv' 's3://<S3 bucket name>/data/graph/'
!aws s3 cp 'data/edge3.csv' 's3://<S3 bucket name>/data/graph/'

### マネジメントコンソールからNeptuneにS3へアクセスするIAMロールを付与する
マネジメントコンソールに戻って操作する。  
元の手順にガイドがあるので、そちらに従って操作を行います。

### curlコマンドを利用してRESTでロード処理を起動

Jupyter Notebook内からREST経由でNeptuneのグラフデータを操作することができる  

"&lt;neptune-endpoint&gt;" をハンズオン環境ごとのEndpoint名で置き換える  
CloudFormationのサービス画面からスタック「neptune」→出力タブで、Endpoint名が確認できる

In [103]:
!curl -X POST -d '{"gremlin":"g.V().count()"}' https://<neptune-endpoint>:8182/gremlin


{"requestId":"eab81c1c-8602-8d4d-8538-d80dd0be0957","status":{"message":"","code":200,"attributes":{"@type":"g:Map","@value":[]}},"result":{"data":{"@type":"g:List","@value":[{"@type":"g:Int64","@value":0}]},"meta":{"@type":"g:Map","@value":[]}}}

APIのレスポンスそのままだと人間が理解しにくいので、出力を整形する簡単な関数を定義する

In [89]:
import ast

def parse_curl_output(output):
    response_dict = ast.literal_eval(output[-1])
    print('Request ID:', response_dict['requestId'])
    print('http response code:', response_dict['status']['code'])
    print('Result:', response_dict['result']['data']['@value'])

既存のデータと重複しないように、既存のグラフデータをいったん削除する

In [91]:
curl_output = !curl -X POST -d '{"gremlin":"g.V().drop()"}' https://<neptune-endpoint>:8182/gremlin
parse_curl_output(curl_output)    

Request ID: 5ab81c0a-5c7c-0d8f-25fa-236e3e41f453
http response code: 200
Result: []


削除後に頂点の数をカウントすると0件であることが確認できる

In [92]:
curl_output = !curl -X POST -d '{"gremlin":"g.V().count()"}' https://<neptune-endpoint>:8182/gremlin
parse_curl_output(curl_output)    

Request ID: d8b81c0a-7c45-648b-43f3-b8c5684351e3
http response code: 200
Result: [{'@type': 'g:Int64', '@value': 0}]


同じようにcurlコマンドからロード処理を実行する

"&lt;S3 bucket name&gt;" , "&lt;neptune-endpoint&gt;" , "&lt;Arn of IAM Role for S3 loading&gt;" をハンズオン環境ごとの値で置き換える  
CloudFormationのサービス画面からスタック「neptune」→出力タブで確認できる

In [106]:
!curl -X POST \
    -H 'Content-Type: application/json' \
    https://<neptune-endpoint>:8182/loader -d ' \
    { \
      "source" : "s3://<S3 bucket name>/data/graph", \
      "format" : "csv", \
      "iamRoleArn" : "<Arn of IAM Role for S3 loading>, \
      "region" : "ap-northeast-1", \
      "failOnError" : "FALSE", \
      "parallelism" : "MEDIUM", \
      "updateSingleCardinalityProperties" : "FALSE" \
    }' \

{
    "status" : "200 OK",
    "payload" : {
        "loadId" : "b882a574-8c54-472b-988a-fa0c4491dac9"
    }
}

ロードコマンドが返却したloadIdを利用してステータスを照会する

"&lt;neptune-endpoint&gt;" をハンズオン環境ごとの値で置き換える  
CloudFormationのサービス画面からスタック「neptune」→出力タブで確認できる

"&lt;LoadId&gt;" はロード処理セルの出力で置き換える

In [100]:
!curl -X GET 'https://<neptune-endpoint>:8182/loader/<LoadId>?details=true&errors=true'

{
    "status" : "200 OK",
    "payload" : {
        "feedCount" : [
            {
                "LOAD_COMPLETED" : 6
            }
        ],
        "overallStatus" : {
            "fullUri" : "s3://tuki4-base-bucket/data/graph",
            "runNumber" : 3,
            "retryNumber" : 0,
            "status" : "LOAD_COMPLETED",
            "totalTimeSpent" : 15,
            "startTime" : 1581490885,
            "totalRecords" : 2026,
            "totalDuplicates" : 0,
            "parsingErrors" : 0,
            "datatypeMismatchErrors" : 0,
            "insertErrors" : 0
        },
        "errors" : {
            "startIndex" : 0,
            "endIndex" : 0,
            "loadId" : "c4ded82f-a009-4799-b5b5-e1d0abb74296",
            "errorLogs" : [ ]
        }
    }
}

ロード完了後に改めて頂点の数をカウントする  
結果が350件になっていれば正常に投入されている

In [101]:
curl_output = !curl -X POST -d '{"gremlin":"g.V().count()"}' https://<neptune-endpoint>:8182/gremlin
parse_curl_output(curl_output)    

Request ID: ccb81c1a-2c55-fc1c-bfab-e6b0e8dfe996
http response code: 200
Result: [{'@type': 'g:Int64', '@value': 350}]


### ここまでの操作でNeptuneにグラフデータが投入できたので、元の手順に戻って「3. Graphexpを利用してグラフデータを可視化する」を進めてください。

### この先のステップは、「3. Graphexpを利用してグラフデータを可視化する」が終わってから実施してください。

### グラフを変更して見え方の違いを確認する

In [23]:
# 事前処理を実施
df_new_edge = 

In [ ]:
# エッジを生成
edge_ids = ['new_edge{}'.format(x) for x in df_new_edge.index.tolist()]
from_nodes = 
to_nodes = 
labels = ['new_edge' for x in df_new_edge.index.tolist()]

df_new_edge_out = pd.DataFrame({'~id': edge_ids, '~from': from_nodes, '~to': to_nodes, '~label': labels})
df_new_edge_out.to_csv('data/edge4.csv', index=False)

エッジファイルをS3にコピーする

In [ ]:
!aws s3 cp 'data/edge4.csv' 's3://tuki4-base-bucket/data/graph2/'

Neptune内にあるエッジ数をカウント

In [105]:
curl_output = !curl -X POST -d '{"gremlin":"g.E().count()"}' https://<neptune-endpoint>:8182/gremlin
parse_curl_output(curl_output)    

Request ID: 4eb81c1f-8cf4-57f7-d3aa-d7e7d4d281a8
http response code: 200
Result: [{'@type': 'g:Int64', '@value': 0}]


エッジファイルをNeptuneにロードする

In [99]:
!curl -X POST \
    -H 'Content-Type: application/json' \
    https://<neptune-endpoint>:8182/loader -d ' \
    { \
      "source" : "s3://<S3 bucket name>/data/graph2", \
      "format" : "csv", \
      "iamRoleArn" : "<Arn of IAM Role for S3 loading>", \
      "region" : "ap-northeast-1", \
      "failOnError" : "FALSE", \
      "parallelism" : "MEDIUM", \
      "updateSingleCardinalityProperties" : "FALSE" \
    }' \



{
    "status" : "200 OK",
    "payload" : {
        "loadId" : "c4ded82f-a009-4799-b5b5-e1d0abb74296"
    }
}

In [100]:
!curl -X GET 'https://<neptune-endpoint>:8182/loader/<LoadId>?details=true&errors=true'

{
    "status" : "200 OK",
    "payload" : {
        "feedCount" : [
            {
                "LOAD_COMPLETED" : 6
            }
        ],
        "overallStatus" : {
            "fullUri" : "s3://tuki4-base-bucket/data/graph",
            "runNumber" : 3,
            "retryNumber" : 0,
            "status" : "LOAD_COMPLETED",
            "totalTimeSpent" : 15,
            "startTime" : 1581490885,
            "totalRecords" : 2026,
            "totalDuplicates" : 0,
            "parsingErrors" : 0,
            "datatypeMismatchErrors" : 0,
            "insertErrors" : 0
        },
        "errors" : {
            "startIndex" : 0,
            "endIndex" : 0,
            "loadId" : "c4ded82f-a009-4799-b5b5-e1d0abb74296",
            "errorLogs" : [ ]
        }
    }
}

エッジが増えていることを確認する

In [105]:
curl_output = !curl -X POST -d '{"gremlin":"g.E().count()"}' https://<neptune-endpoint>:8182/gremlin
parse_curl_output(curl_output)    

Request ID: 4eb81c1f-8cf4-57f7-d3aa-d7e7d4d281a8
http response code: 200
Result: [{'@type': 'g:Int64', '@value': 0}]


### WorkSpacesのブラウザ（Graphexp）に戻って、新しいグラフの見え方を確認してください